# Frac Production Modeling Multiple States

This is a continuation of Frac Production Modeling for Colorado.  Here I will combine the data with other states and model the data.

In [1]:
# Necessary Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Model Imports
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

Great, now I will write two functions to streamline training and testing the models.  The training function will fit the model and then cross validate the scores and return the scores.  The test function will cross validate and return the scores.

In [2]:
def train_model(model, X, Y, cv):
    model.fit(X, Y)
    scores = cross_val_score(model, X, Y, cv=cv)
    return 'Training Scores: {:0.4f} (+/- {:0.4f})'.format(scores.mean(), scores.std()*2)

def test_model(model, X, Y, cv):
    scores = cross_val_score(model, X, Y, cv=cv)
    return 'Test Scores: {:0.4f} (+/- {:0.4f})'.format(scores.mean(), scores.std()*2)


### Colorado Data

In [3]:
# Reading in only engineered Colorado Features
cofeatures = pd.read_csv('cofeatures.csv', index_col=0,usecols=['APINumber','VerticalDepth','First6BOE', 'nphf_sqrt', 
                                                            'slick','gel','xlinkgel','sqrtsandmass',#'hybrid',
                                                            'hzlen_bin_1-2','hzlen_bin_<1','hzlen_bin_>2'])
cofeatures.head()

,gel,slick,xlinkgel,VerticalDepth,nphf_sqrt,sqrtsandmass,First6BOE,hzlen_bin_<1,hzlen_bin_1-2,hzlen_bin_>2
APINumber,,,,,,,,,,
5001098010000,0,1,1,7774.00,73.120155,1591.816874,46241.0,1,0,0
5001097820000,0,1,1,7574.72,73.519946,1668.595385,7118.0,1,0,0
5001098410000,0,0,0,8045.00,73.509025,1772.463745,23404.0,1,0,0
5001098450000,0,1,0,7841.00,73.543258,1999.739152,97243.0,1,0,0
5001098470000,0,1,0,7707.00,73.740393,1984.574673,93034.0,1,0,0


Knowing that this data is clean, I will first identify my variables, with the first 6 month's production as the target and the remainder as features in the model.  I will then split both variables into training and test sets.

In [4]:
# Identifying all engineered features
cofeats = cofeatures.drop(['First6BOE'], axis=1)
print(cofeats.shape)
cotarget = cofeatures.First6BOE

(3831, 9)


Great, now let's load in other state data.

### Oklahoma Data

In [5]:
# Reading in Oklahoma features
okfeatures = pd.read_csv('okfeatures', index_col=0).dropna()
# Dropping features not in Colorado data
okfeatures = okfeatures.drop(['SandMassPerFoot','hybrid'], axis=1)
print(okfeatures.shape)
okfeatures.head()

(10358, 10)


,gel,slick,xlinkgel,VerticalDepth,nphf_sqrt,sqrtsandmass,First6BOE,hzlen_bin_<1,hzlen_bin_1-2,hzlen_bin_>2
APINumber,,,,,,,,,,
35051238300000,0,1,0,12246.0,79.170702,1839.189767,62733.0,1,0,0
42483338530000,0,1,0,13487.0,77.038951,2032.532411,132965.0,1,0,0
42375318560000,0,1,0,9335.0,79.328431,2796.000894,155981.0,1,0,0
35043232790000,0,1,0,8968.0,78.025637,1740.442472,10426.0,1,0,0
35149215590000,0,1,0,12735.0,77.498387,1760.266741,51461.0,1,0,0


Since there are two prefix numbers for the API numbers, this means that there are actually two states in this data set.  State prefix 35 is for Oklahoma, and state prefix 42 is Texas.

In [6]:
# Splitting out Oklahoma wells
okwellfeatures = okfeatures[okfeatures.index <40000000000000]
print(okwellfeatures.shape)
# Splitting out Texas wells
txwellfeatures = okfeatures[okfeatures.index >40000000000000]
print(txwellfeatures.shape)

(7904, 10)
(2454, 10)


In [7]:
# Identifying variables for Oklahoma
oktarget = okwellfeatures.First6BOE
okfeats = okwellfeatures.drop('First6BOE', axis=1)
print(okfeats.shape)

# Identifying variables for Texas
txtarget = txwellfeatures.First6BOE
txfeats = txwellfeatures.drop('First6BOE', axis=1)
print(txfeats.shape)

(7904, 9)
(2454, 9)


### Montana Data

In [10]:
# Reading in Montana data
mtfeatures = pd.read_csv('mtfeatures.csv', index_col=0).dropna()
# Dropping feature not in all data
mtfeatures = mtfeatures.drop(['hybrid'], axis=1)
print(mtfeatures.shape)
mtfeatures.head()

(9633, 10)


,gel,slick,xlinkgel,VerticalDepth,nphf_sqrt,sqrtsandmass,First6BOE,hzlen_bin_<1,hzlen_bin_1-2,hzlen_bin_>2
APINumber,,,,,,,,,,
25085219780000,0,1,0,10071.91,119.565811,3540.964318,112335.67,0,0,1
25085219820000,0,1,0,10149.72,119.890755,3482.759695,103295.83,0,0,1
25085219830000,0,1,0,10185.45,120.039673,3211.662658,109993.67,0,0,1
33025020720000,0,1,1,10775.58,116.429349,1614.653077,36293.00,0,0,1
33025020730000,0,1,1,10849.07,121.762282,1333.051429,27982.17,0,0,1


In [11]:
# Identifying features for Montana
mttarget = mtfeatures.First6BOE
mtfeats = mtfeatures.drop('First6BOE', axis=1)
print(mtfeats.shape)

(9633, 9)


Great! Now that we have all of our data and variables identified, I will concatenate them together.

### Concatenating Data

In [12]:
# Concatenating all features together
all_states = pd.concat([cofeats, okfeats, txfeats, mtfeats], axis=0)
print(all_states.shape)

# Concatenating all targets together
all_targets = pd.concat([cotarget, oktarget, txtarget, mttarget], axis=0)
print(all_targets.shape)
all_states.tail()

(23822, 9)
(23822,)


,gel,slick,xlinkgel,VerticalDepth,nphf_sqrt,sqrtsandmass,hzlen_bin_<1,hzlen_bin_1-2,hzlen_bin_>2
APINumber,,,,,,,,,
33025020670000,0,1,1,10994.51,124.016181,1812.579017,0,0,1
33025020680000,0,1,1,11157.81,118.074058,1827.365371,0,0,1
33025020690000,0,1,1,11103.75,117.983826,1818.890189,0,0,1
33025020700000,0,1,1,10981.19,118.985181,1332.265340,0,0,1
33025020710000,0,1,1,10969.89,119.010559,1328.820965,0,0,1


Alright, now that I have all of the data together, I need to split that into training and test sets.  I will keep out a test size of 25%.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(all_states, all_targets, test_size=0.25, random_state=42)

## Modeling
### Random Forest Regressor
I will start with a simple random forest model to determine how well this data predicts production for all four states.

In [14]:
# Instantiate the model
rfr = RandomForestRegressor()

# Fit the model and generate training scores
rfr_train = train_model(rfr, X_train, y_train, 5)
# Generate test scores
rfr_test = test_model(rfr, X_test, y_test, 5)
print(rfr_train)
print(rfr_test)

Training Scores: 0.2915 (+/- 0.0391)
Test Scores: 0.2688 (+/- 0.0518)


The good thing about this result is that the variance in the test set was greatly reduced, likely because of the increase in number of samples, however the model was only able to explain the variance in 27% of the samples, with an R-squared of 0.27 +/- 0.05.

### Random Forest Quantile Regressor
Now since the quantilization of the data was able to explain the most variance in the Colorado data, I will try the same for all states.  I will start with the default values.

In [15]:
from skgarden import RandomForestQuantileRegressor

rfqr = RandomForestQuantileRegressor(random_state=42)
rfqr_train = train_model(rfqr, X_train, y_train, 5)
rfqr_test = test_model(rfqr, X_test, y_test, 5)
print(rfqr_train)
print(rfqr_test)

Training Scores: 0.2849 (+/- 0.0438)
Test Scores: 0.2750 (+/- 0.0702)


Well, that's a slight improvement in R-squared score.  Let's try and optimize the parameters of this model, including the number of estimators, the maximum number of features to use, the minimum samples split, and the maximum depth of the tree.

In [16]:
# Identifying potential parameters
param_grid = { 
            "n_estimators"      : [10, 100, 1000],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,6,8],
            "max_depth": [4,6,8,10]
            }
# Instantiating grid search
grid = GridSearchCV(estimator=rfqr, param_grid=param_grid, cv=5)
# Fitting model
grid.fit(X_train, y_train)
# Identifying best score and best parameters from the Grid Search
print(grid.best_score_)
best_params = grid.best_params_
print(best_params)

0.3461033013515488
{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 1000}


Using these optimal parameters, let's now apply those parameters to a new model.

In [24]:
# Instantiate the model
rfqr_grid = RandomForestQuantileRegressor(**best_params, random_state=42)
# Fit the model and generate training scores
rfqr_grid_train = train_model(rfqr_grid, X_train, y_train, 5)
# Generate test scores
rfqr_grid_test = test_model(rfqr_grid, X_test, y_test, 5)
print(rfqr_grid_train)
print(rfqr_grid_test)

Training Scores: 0.3461 (+/- 0.0336)
Test Scores: 0.3437 (+/- 0.0391)


Again, we have a fairly low variance, and a relatively higher R-squared score from the previous models with all of the states.

### Model Transferance Across States 
Initially I used Colorado well data to predict other wells in Colorado.  Now I will use all of the Colorado data to predict the other state's production, and then use the other states to predict Colorado production.

In [25]:
state_list = [['Oklahoma', okfeats, oktarget],
             ['Texas', txfeats, txtarget],
             ['Montana', mtfeats, mttarget]]
# Iterating over states for Colorado to predict
for states in state_list:
    print('Colorado Predicting {}'.format(states[0]))
    # Fit the model and generate training scores
    rfqr_grid = RandomForestQuantileRegressor(**best_params, random_state=42)
    rfqr_train = train_model(rfqr_grid, cofeats, cotarget, 5)
    # Generate test scores
    rfqr_test = test_model(rfqr_grid, states[1], states[2], 5)
    print(rfqr_train)
    print(rfqr_test)
    print()

# Iterating over states to predict Colorado
for states in state_list:
    print('{} Predicting Colorado'.format(states[0]))
    # Fit the model and generate training scores
    rfqr_grid = RandomForestQuantileRegressor(**best_params, random_state=42)
    rfqr_train = train_model(rfqr_grid, states[1], states[2], 5)
    # Generate test scores
    rfqr_test = test_model(rfqr_grid, cofeats, cotarget, 5)
    print(rfqr_train)
    print(rfqr_test)
    print()

Colorado Predicting Oklahoma
Training Scores: 0.4280 (+/- 0.2064)
Test Scores: 0.3509 (+/- 0.2050)

Colorado Predicting Texas
Training Scores: 0.4280 (+/- 0.2064)
Test Scores: 0.2588 (+/- 0.0678)

Colorado Predicting Montana
Training Scores: 0.4280 (+/- 0.2064)
Test Scores: 0.0650 (+/- 0.1211)

Oklahoma Predicting Colorado
Training Scores: 0.3509 (+/- 0.2050)
Test Scores: 0.4280 (+/- 0.2064)

Texas Predicting Colorado
Training Scores: 0.2588 (+/- 0.0678)
Test Scores: 0.4280 (+/- 0.2064)

Montana Predicting Colorado
Training Scores: 0.0650 (+/- 0.1211)
Test Scores: 0.4280 (+/- 0.2064)



Colorado best predicted Oklahoma's production with an R-squared value of 0.35 +/- 0.2, however Colorado was able to predict Texas well production with a third of the variance (R-squared = 0.26 +/- 0.068).

All states predicting Colorado's production gave identical R-squared values of 0.43 +/- 0.2.  

## Leave One State Out
Now, since there are differences in oil and gas production across states, I will use three states to try and predict the production of the fourth.

In [26]:
oktxmtfeats = pd.concat([okfeats, txfeats, mtfeats], axis=0)
oktxmttarget = pd.concat([oktarget, txtarget, mttarget], axis=0)

okcomtfeats = pd.concat([okfeats, cofeats, mtfeats], axis=0)
okcomttarget = pd.concat([oktarget, cotarget, mttarget], axis=0)

cotxmtfeats = pd.concat([cofeats, txfeats, mtfeats], axis=0)
cotxmttarget = pd.concat([cotarget, txtarget, mttarget], axis=0)

cotxokfeats = pd.concat([cofeats, txfeats, okfeats], axis=0)
cotxoktarget = pd.concat([cotarget, txtarget, oktarget], axis=0)

print('Oklahoma, Texas, and Montana Predicting Colorado')
# Fit the model and generate training scores
rfqr_grid = RandomForestQuantileRegressor(**best_params, random_state=42)
rfqr_train = train_model(rfqr_grid, oktxmtfeats, oktxmttarget, 5)
# Generate test scores
rfqr_test = test_model(rfqr_grid, cofeats, cotarget, 5)
print(rfqr_train)
print(rfqr_test)
print()

print('Oklahoma, Colorado, and Montana Predicting Texas')
# Fit the model and generate training scores
rfqr_grid = RandomForestQuantileRegressor(**best_params, random_state=42)
rfqr_train = train_model(rfqr_grid, okcomtfeats, okcomttarget, 5)
# Generate test scores
rfqr_test = test_model(rfqr_grid, txfeats, txtarget, 5)
print(rfqr_train)
print(rfqr_test)
print()

print('Colorado, Texas, and Montana Predicting Oklahoma')
# Fit the model and generate training scores
rfqr_grid = RandomForestQuantileRegressor(**best_params, random_state=42)
rfqr_train = train_model(rfqr_grid, cotxmtfeats, cotxmttarget, 5)
# Generate test scores
rfqr_test = test_model(rfqr_grid, okfeats, oktarget, 5)
print(rfqr_train)
print(rfqr_test)
print()

print('Colorado, Texas, and Oklahoma Predicting Montana')
# Fit the model and generate training scores
rfqr_grid = RandomForestQuantileRegressor(**best_params, random_state=42)
rfqr_train = train_model(rfqr_grid, cotxokfeats, cotxoktarget, 5)
# Generate test scores
rfqr_test = test_model(rfqr_grid, mtfeats, mttarget, 5)
print(rfqr_train)
print(rfqr_test)
print()

Oklahoma, Texas, and Montana Predicting Colorado
Training Scores: 0.1368 (+/- 0.1164)
Test Scores: 0.4280 (+/- 0.2064)

Oklahoma, Colorado, and Montana Predicting Texas
Training Scores: 0.1453 (+/- 0.1639)
Test Scores: 0.2588 (+/- 0.0678)

Colorado, Texas, and Montana Predicting Oklahoma
Training Scores: 0.1519 (+/- 0.3066)
Test Scores: 0.3509 (+/- 0.2050)

Colorado, Texas, and Oklahoma Predicting Montana
Training Scores: 0.3191 (+/- 0.2012)
Test Scores: 0.0650 (+/- 0.1211)



According to these models, Oklahoma, Texas, and Montana were best able to predict Colorado's production, with an R-squared value of 0.43 +/- 0.2, however Oklahoma, Colorado, and Montana had the lowest variance in R-squared scores in predicting Texas's production, with an R-squared value of 0.2588 +/- 0.07.

## State by State Modeling
Finally, I will check to see how well each of the additional state data are able to predict within the state, again to eliminate the variability across states.

In [27]:
state_list = [['Oklahoma', okfeats, oktarget],
             ['Texas', txfeats, txtarget],
             ['Montana', mtfeats, mttarget]]

for states in state_list:
    print('Predicting {}'.format(states[0]))
    # Fit the model and generate training scores
    Xs_train, Xs_test, ys_train, ys_test = train_test_split(states[1], states[2], test_size=0.25, random_state=42)
    rfqr_grid = RandomForestQuantileRegressor(**best_params, random_state=42)
    rfqr_train = train_model(rfqr_grid, Xs_train, ys_train, 5)
    # Generate test scores
    rfqr_test = test_model(rfqr_grid, Xs_test, ys_test, 5)
    print(rfqr_train)
    print(rfqr_test)
    print()

Predicting Oklahoma
Training Scores: 0.4616 (+/- 0.0470)
Test Scores: 0.4187 (+/- 0.0888)

Predicting Texas
Training Scores: 0.2339 (+/- 0.0613)
Test Scores: 0.1452 (+/- 0.1825)

Predicting Montana
Training Scores: 0.2450 (+/- 0.0388)
Test Scores: 0.2261 (+/- 0.0743)



From this analysis, these features are best able to predict the production in Oklahoma, with a R-squared score of 0.42 +/- 0.088.  These features were least able to predict Texas production, with the variance being larger than the R-squared value.

This could go back to data quality because the Oklahoma data had the frac information in the original data set, and did not have to be calculated and scraped from the fracfocus data.  Interestingly enough, the Texas data was also a part of that data set, yet it has dramatically lower R-squared scores.  This could be because Texas production differs in its methods or correlations to higher production.